In [1]:
import pandas as pd
from contextlib import chdir
import os
import pathlib

In [2]:
b3lyp = pd.read_csv('b3lyp_eprec6.csv')
hf = pd.read_csv('hf_eprec6.csv')
lda = pd.read_csv('lda_eprec6.csv')
pbe0 = pd.read_csv('pbe0_eprec6.csv')

In [3]:
print(pbe0['Unnamed: 0'])
print(lda['Unnamed: 0'])

0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object
0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object


In [4]:
print(pbe0['Energy'])

0     -234.448220
1     -343.028857
2     -412.752433
3      -82.619718
4     -113.237996
          ...    
220   -116.526064
221   -157.084428
222   -290.496811
223   -242.867793
224    -89.829275
Name: Energy, Length: 225, dtype: float64


In [5]:
#print(pbe0)

In [6]:
#with chdir("/home/hannes/Documents/SBU/AMS561/ML/geometries"):
path = os.path.abspath(os.getcwd())
geometries_path = os.path.join(path, 'geometries')
mols = {}
#for filename in os.scandir("/home/hannes/Documents/SBU/AMS561/ML/geometries"):
for filename in os.scandir(geometries_path):
    name = str(filename).split()[1]
    name = name.replace('.mdns', '')
    name = name.replace("'", "")
    name = name.replace(">", "")
    if filename.is_file():
        f = open(filename, "r")
        mol = []
        while True:
            line = f.readline()
            if "geometry" in line:
                line = f.readline()
                if "units angstrom" in line:
                    f.readline()
                while True:
                    line = f.readline()
                    if line == "end":
                        break
                    mol.append(line.split())
                mols[name] = mol        
            if not line:
                break

In [7]:
print(len(mols))

226


In [8]:
print(mols["water"])

[['H', '0.000000000000', '0.757200000000', '-0.469200000000'], ['H', '0.000000000000', '-0.757200000000', '-0.469200000000']]


In [9]:
for i in mols["bicyclo[3.1.0]hexane"]:
    print(i)

['C', '0.0000000', '0.7642750', '1.2644900']
['C', '0.0000000', '-0.7642750', '1.2644900']
['C', '-0.7642750', '0.0000000', '-1.2644900']
['C', '0.7642750', '0.0000000', '-1.2644900']
['H', '0.9134530', '1.2576380', '1.5632730']
['H', '-0.9134530', '1.2576380', '1.5632730']
['H', '-0.9134530', '-1.2576380', '1.5632730']
['H', '0.9134530', '-1.2576380', '1.5632730']
['H', '-1.2576380', '-0.9134530', '-1.5632730']
['H', '-1.2576380', '0.9134530', '-1.5632730']
['H', '1.2576380', '0.9134530', '-1.5632730']
['H', '1.2576380', '-0.9134530', '-1.5632730']
